# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 5034, uptime 0:00:16
mini_beamline                    RUNNING   pid 5035, uptime 0:00:16
random_walk                      RUNNING   pid 5036, uptime 0:00:16
random_walk_horiz                RUNNING   pid 5050, uptime 0:00:14
random_walk_vert                 RUNNING   pid 5038, uptime 0:00:16
simple                           RUNNING   pid 5039, uptime 0:00:16
thermo_sim                       RUNNING   pid 5040, uptime 0:00:16
trigger_with_pc                  RUNNING   pid 5041, uptime 0:00:16


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-15 18:02:33
Persistent Unique Scan ID: '5e1a1a7b-e431-4f20-9056-7510806783ca'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 18:02:33.2 |     -1.000 |     102841 |


|         2 | 18:02:33.2 |     -0.500 |     102946 |
|         3 | 18:02:33.3 |      0.000 |     102953 |
|         4 | 18:02:33.3 |      0.500 |     101845 |
|         5 | 18:02:33.3 |      1.000 |     100974 |
+-----------+------------+------------+------------+
generator scan ['5e1a1a7b'] (scan num: 1)





('5e1a1a7b-e431-4f20-9056-7510806783ca',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/5e1a1a7b-e431-4f20-9056-7510806783ca-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-15 18:02:33
Persistent Unique Scan ID: '5920c5d7-82a0-450f-85fa-64f91088646f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 18:02:33.7 |          -1 | 1104363362 |


|         2 | 18:02:33.9 |          -0 | 1094169217 |


|         3 | 18:02:34.0 |           0 | 1081509473 |


|         4 | 18:02:34.1 |           0 | 1074287678 |


|         5 | 18:02:34.2 |           1 | 1068395645 |


+-----------+------------+-------------+------------+
generator scan ['5920c5d7'] (scan num: 2)





('5920c5d7-82a0-450f-85fa-64f91088646f',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-15 18:02:34
Persistent Unique Scan ID: 'c445b9c3-a52d-4ec6-a2db-da1367c85377'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 18:02:34.7 | 1054728259 |


|         2 | 18:02:35.6 | 1085511984 |


|         3 | 18:02:36.6 | 1131192755 |


...

|        28 | 18:03:01.7 | 1096701632 |


|        29 | 18:03:02.7 | 1053445409 |


|        30 | 18:03:03.7 | 1088960400 |


+-----------+------------+------------+
generator count ['c445b9c3'] (scan num: 3)





('c445b9c3-a52d-4ec6-a2db-da1367c85377',)